In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.6MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [5]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [12]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(feats, model):
  X = df[cat_feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## DecisionTree


In [16]:
run_model(cat_feats, DecisionTreeRegressor(max_depth=5))

(-19747.81093847179, 75.95212671816309)

## RandomForest


In [17]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(cat_feats, model)

(-18864.27843495869, 38.24233578175403)

## XGBOOST


In [18]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

run_model(cat_feats, xgb.XGBRegressor(**xgb_params))

[21:54:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:54:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:55:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13034.249470063352, 103.73566521784606)

In [20]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(X, y)


imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

[22:01:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1274 ± 0.0017,param_napęd__cat
0.1159 ± 0.0025,param_stan__cat
0.1131 ± 0.0037,param_rok-produkcji__cat
0.0565 ± 0.0010,param_faktura-vat__cat
0.0482 ± 0.0014,param_moc__cat
0.0416 ± 0.0017,param_skrzynia-biegów__cat
0.0252 ± 0.0010,param_marka-pojazdu__cat
0.0247 ± 0.0014,feature_kamera-cofania__cat
0.0232 ± 0.0008,param_typ__cat
0.0198 ± 0.0006,param_pojemność-skokowa__cat


In [0]:
feats = ['param_napęd__cat',
'param_stan__cat',
'param_rok-produkcji__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_skrzynia-biegów__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'param_wersja__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'feature_system-start-stop__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat',
'feature_czujniki-parkowania-przednie__cat']

In [22]:
run_model(feats, xgb.XGBRegressor(**xgb_params))

[22:12:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:12:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:12:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13034.249470063352, 103.73566521784606)

In [35]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats = ['param_napęd__cat',
'param_stan__cat',
'param_rok-produkcji',
'param_faktura-vat__cat',
'param_moc__cat',
'param_skrzynia-biegów__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'param_wersja__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'feature_system-start-stop__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat',
'feature_czujniki-parkowania-przednie__cat']

run_model(feats, xgb.XGBRegressor(**xgb_params))

[22:26:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:26:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13034.249470063352, 103.73566521784606)

In [37]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

feats = ['param_napęd__cat',
'param_stan__cat',
'param_rok-produkcji',
'param_faktura-vat__cat',
'param_moc',
'param_skrzynia-biegów__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'param_wersja__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'feature_system-start-stop__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat',
'feature_czujniki-parkowania-przednie__cat']

run_model(feats, xgb.XGBRegressor(**xgb_params))

[22:28:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13034.249470063352, 103.73566521784606)